In [2]:
import numpy as np 
import pandas as pd 
# import os
import re
import itertools
import unicodedata
import networkx as nx
from scipy.spatial import distance
%matplotlib inline
from matplotlib import pyplot as plt

import nltk
from nltk import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk import bigrams

import collections

In [3]:
# Data Pre-processing ==> done in text pre-processing docs
# Import data 
data = pd.read_csv('Reddit Stuff/merged_clean.csv')
data

data_list = data['Processed'].tolist()

In [28]:
# tokenise words in each row and put them in a list. create a list of lists
tokenizer = RegexpTokenizer(r'\w+')

reddit_words = []
for reddit_text in data_list:
    if type(reddit_text) == type("string"):
        tokens = tokenizer.tokenize(reddit_text)
        reddit_words.append(tokens)

reddit_words_list = list(itertools.chain(*reddit_words))        
print(len(reddit_words))
print(len(reddit_words_list))

87475
740176


In [29]:
reddit_words_list = list(itertools.chain(*reddit_words))
reddit_words_counts = collections.Counter(reddit_words_list)
# {k: v for k, v in sorted(reddit_words_counts.items(), key=lambda item: item[1])}
# reddit_words_counts

to_remove=[]
for k, v in reddit_words_counts.items():
    if v <= 5:
        to_remove.append(k)
len(to_remove)

# #  4 -> 25797
# #  5 -> 26845
# #  10 -> 29626

26845

In [33]:
to_remove.sort()
to_remove

['0000',
 '00000',
 '000000',
 '00000000',
 '00000000000',
 '000000000000001',
 '0000000001',
 '000000001',
 '0000000016',
 '0000000018',
 '000000002',
 '0000000025',
 '000000003',
 '00000000408',
 '0000000050',
 '0000000055',
 '000000006',
 '000000008',
 '0000000096',
 '00000000xxxx',
 '0000000103',
 '000000011',
 '0000000126209',
 '000000016178',
 '0000000180',
 '00000002',
 '00000003',
 '000000035',
 '00000004',
 '00000008',
 '00000009',
 '0000001',
 '0000002',
 '0000003',
 '000000413270',
 '0000005',
 '000001095',
 '000005',
 '000008',
 '000009',
 '00000yyy',
 '00005',
 '00009',
 '00009876',
 '00015',
 '0002',
 '0005',
 '0005eth',
 '00076',
 '0008',
 '0009',
 '000k',
 '000s',
 '00100000',
 '00111000',
 '0012',
 '0014',
 '001470',
 '001bnb',
 '001eth',
 '0025',
 '0026',
 '0027',
 '0029',
 '002950',
 '002eth',
 '0035',
 '003eth',
 '004',
 '0042',
 '0046',
 '0056',
 '006',
 '007',
 '0073',
 '008ashah',
 '009eth',
 '00am',
 '00cest',
 '00utc',
 '010',
 '0100',
 '0101',
 '010101010',
 '

In [56]:
# include stopwords if needed 
stopwords_LDA = ['12', '24', '6', '4', 'cardano', '2', '9', '0', '00','000', '000001', '00001', '1', '01', '0100', 
'01100001', '01100101', '01101101', '01101110', '01101111', '01110011', '0147', '015', '0172','02']
stopwords_LDA = stopwords_LDA + to_remove
for i in reddit_words:
    for j in i:
        if j in stopwords_LDA:
            i.remove(j)
print("done")

done


In [55]:
# stopwords_LDA = ['12', '24', '6', '4', 'cardano', '2', '9', '0', '00','000', '000001', '00001', '1', '01', '0100', '01100001', '01100101', '01101101', '01101110', '01101111', '01110011', '0147', '015', '0172','02']
# stopwords_LDA = stopwords_LDA + to_remove
# stopwords_LDA

['12',
 '24',
 '6',
 '4',
 'cardano',
 '2',
 '9',
 '0',
 '00',
 '000',
 '000001',
 '00001',
 '1',
 '01',
 '0100',
 '01100001',
 '01100101',
 '01101101',
 '01101110',
 '01101111',
 '01110011',
 '0147',
 '015',
 '0172',
 '02',
 '0000',
 '00000',
 '000000',
 '00000000',
 '00000000000',
 '000000000000001',
 '0000000001',
 '000000001',
 '0000000016',
 '0000000018',
 '000000002',
 '0000000025',
 '000000003',
 '00000000408',
 '0000000050',
 '0000000055',
 '000000006',
 '000000008',
 '0000000096',
 '00000000xxxx',
 '0000000103',
 '000000011',
 '0000000126209',
 '000000016178',
 '0000000180',
 '00000002',
 '00000003',
 '000000035',
 '00000004',
 '00000008',
 '00000009',
 '0000001',
 '0000002',
 '0000003',
 '000000413270',
 '0000005',
 '000001095',
 '000005',
 '000008',
 '000009',
 '00000yyy',
 '00005',
 '00009',
 '00009876',
 '00015',
 '0002',
 '0005',
 '0005eth',
 '00076',
 '0008',
 '0009',
 '000k',
 '000s',
 '00100000',
 '00111000',
 '0012',
 '0014',
 '001470',
 '001bnb',
 '001eth',
 '0025',


In [57]:
print(len(reddit_words))
reddit_words_list = list(itertools.chain(*reddit_words))
print(len(reddit_words_list))

87475
685504


In [59]:
reddit_dict = {}

for post in reddit_words:
    count = 0
    for i in range(count,len(post)):
        count += 1
        for j in range(count,len(post)):
            if (post[i],post[j]) in reddit_dict:
                reddit_dict[(post[i],post[j])] += 1
                
            elif (post[j],post[i]) in reddit_dict:
                reddit_dict[(post[j],post[i])] += 1
                
            else:
                reddit_dict[(post[i],post[j])] = 1

reddit_dict

{('nft', 'marketplace'): 2513,
 ('nft', 'projects'): 1044,
 ('nft', 'updated'): 50,
 ('nft', 'genesis'): 159,
 ('nft', 'house'): 214,
 ('nft', 'artano'): 59,
 ('nft', 'curated'): 34,
 ('nft', 'community'): 1838,
 ('nft', 'high'): 544,
 ('nft', 'quality'): 233,
 ('nft', 'nft'): 7170,
 ('nft', 'cares'): 16,
 ('nft', 'inclusivity'): 13,
 ('nft', 'jpg'): 25,
 ('nft', 'store'): 600,
 ('nft', 'trade'): 326,
 ('nft', 'nfts'): 9031,
 ('nft', 'instantly'): 76,
 ('nft', 'discover'): 44,
 ('nft', 'artwork'): 578,
 ('nft', 'explore'): 52,
 ('nft', 'communities'): 81,
 ('nft', 'support'): 444,
 ('nft', 'artists'): 1075,
 ('nft', 'cnft'): 196,
 ('nft', 'io'): 239,
 ('nft', 'digital'): 835,
 ('nft', 'token'): 2354,
 ('nft', 'platform'): 1365,
 ('nft', 'lovada'): 33,
 ('nft', 'inclusive'): 44,
 ('nft', 'blockchain'): 1476,
 ('nft', 'supporting'): 54,
 ('nft', 'artist'): 920,
 ('nft', 'art'): 3022,
 ('nft', 'market'): 1290,
 ('nft', 'ada'): 456,
 ('nft', 'backs'): 23,
 ('nft', 'creators'): 849,
 ('nft'

In [75]:
len(reddit_dict)

2988701

In [77]:
newreddit = []

for i in reddit_dict:
    if reddit_dict[i] > 15:   
        temp = []
        temp.append(i[0])
        temp.append(i[1])
        temp.append(reddit_dict[i])
        newreddit.append(temp)

newreddit  

[['nft', 'marketplace', 2513],
 ['nft', 'projects', 1044],
 ['nft', 'updated', 50],
 ['nft', 'genesis', 159],
 ['nft', 'house', 214],
 ['nft', 'artano', 59],
 ['nft', 'curated', 34],
 ['nft', 'community', 1838],
 ['nft', 'high', 544],
 ['nft', 'quality', 233],
 ['nft', 'nft', 7170],
 ['nft', 'cares', 16],
 ['nft', 'jpg', 25],
 ['nft', 'store', 600],
 ['nft', 'trade', 326],
 ['nft', 'nfts', 9031],
 ['nft', 'instantly', 76],
 ['nft', 'discover', 44],
 ['nft', 'artwork', 578],
 ['nft', 'explore', 52],
 ['nft', 'communities', 81],
 ['nft', 'support', 444],
 ['nft', 'artists', 1075],
 ['nft', 'cnft', 196],
 ['nft', 'io', 239],
 ['nft', 'digital', 835],
 ['nft', 'token', 2354],
 ['nft', 'platform', 1365],
 ['nft', 'lovada', 33],
 ['nft', 'inclusive', 44],
 ['nft', 'blockchain', 1476],
 ['nft', 'supporting', 54],
 ['nft', 'artist', 920],
 ['nft', 'art', 3022],
 ['nft', 'market', 1290],
 ['nft', 'ada', 456],
 ['nft', 'backs', 23],
 ['nft', 'creators', 849],
 ['nft', 'craze', 35],
 ['nft', 'goa

In [79]:
G = nx.Graph()
G.add_weighted_edges_from(newreddit)
nx.write_gml(G, "reddit_compiled.gml")

In [61]:
# #Put it into a Dataframe
# bigram_df = pd.DataFrame(reddit_dict.items(), columns=['bigram', 'count'])

In [62]:
# df = bigram_df.copy()

# word1 = []
# word2 = []
# for i, j in df['bigram']:
#     word1.append(i) 
#     word2.append(j)
    
# df['word1']=word1
# df['word2']=word2
# df

,bigram,count,word1,word2
0,"(nft, marketplace)",2513,nft,marketplace
1,"(nft, projects)",1044,nft,projects
2,"(nft, updated)",50,nft,updated
3,"(nft, genesis)",159,nft,genesis
4,"(nft, house)",214,nft,house
...,...,...,...,...
2988696,"(tweet, considered)",1,tweet,considered
2988697,"(pls, sure)",1,pls,sure
2988698,"(pls, considered)",1,pls,considered
2988699,"(joined, considered)",1,joined,considered


In [63]:
# # remove those who have a count less than 5 - removed 4062560
# # remove those who have a count less than 4 - 3946132
# to_remove=[]
# for x in range(len(bigram_df['count'])):
#     if bigram_df.iloc[x][1] <= 5:
#         to_remove.append(x)
# len(to_remove)
# bigram_df.drop(to_remove, axis=0, inplace=True)

In [66]:
df2 = df.copy()

In [67]:
df2.drop(to_remove, axis=0, inplace=True)
df2

,bigram,count,word1,word2
0,"(nft, marketplace)",2513,nft,marketplace
1,"(nft, projects)",1044,nft,projects
2,"(nft, updated)",50,nft,updated
3,"(nft, genesis)",159,nft,genesis
4,"(nft, house)",214,nft,house
...,...,...,...,...
2988248,"(sell, script)",6,sell,script
2988255,"(seconds, script)",6,seconds,script
2988267,"(perfect, script)",6,perfect,script
2988280,"(tool, script)",6,tool,script


In [70]:
list_comp = []
for x in range(len(df2['count'])):
    if df2.iloc[x][1] > 5:
        to_add = [df2.iloc[x][2], df2.iloc[x][3], str(df2.iloc[x][1])]
        list_comp.append(to_add)

In [71]:
len(list_comp)

501303

In [72]:
list_comp

[['nft', 'marketplace', '2513'],
 ['nft', 'projects', '1044'],
 ['nft', 'updated', '50'],
 ['nft', 'genesis', '159'],
 ['nft', 'house', '214'],
 ['nft', 'artano', '59'],
 ['nft', 'curated', '34'],
 ['nft', 'community', '1838'],
 ['nft', 'high', '544'],
 ['nft', 'quality', '233'],
 ['nft', 'nft', '7170'],
 ['nft', 'cares', '16'],
 ['nft', 'inclusivity', '13'],
 ['nft', 'jpg', '25'],
 ['nft', 'store', '600'],
 ['nft', 'trade', '326'],
 ['nft', 'nfts', '9031'],
 ['nft', 'instantly', '76'],
 ['nft', 'discover', '44'],
 ['nft', 'artwork', '578'],
 ['nft', 'explore', '52'],
 ['nft', 'communities', '81'],
 ['nft', 'support', '444'],
 ['nft', 'artists', '1075'],
 ['nft', 'cnft', '196'],
 ['nft', 'io', '239'],
 ['nft', 'digital', '835'],
 ['nft', 'token', '2354'],
 ['nft', 'platform', '1365'],
 ['nft', 'lovada', '33'],
 ['nft', 'inclusive', '44'],
 ['nft', 'blockchain', '1476'],
 ['nft', 'supporting', '54'],
 ['nft', 'artist', '920'],
 ['nft', 'art', '3022'],
 ['nft', 'market', '1290'],
 ['nft'

In [73]:
G = nx.Graph()
G.add_weighted_edges_from(list_comp)
nx.write_gml(G,"reddit_complied_2.gml")

In [74]:
highest_weights = sorted(G.edges(data=True), key=lambda t: t[2].get('weight'),reverse=True)
highest_weights[:3]

[('nft', 'world', {'weight': '997'}),
 ('nfts', 'first', {'weight': '995'}),
 ('nft', 'told', {'weight': '99'})]

References:<br> 
https://www.kaggle.com/code/itoeiji/simple-co-occurrence-network/notebook <br>
https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/calculate-tweet-word-bigrams/